# Analysis of solved instances (Part 1)
Comparing:
1. Zonotope Propagation with LP branching check and LP fallback for equivalence check (`091c775`)
2. Same as above but with CEGAR overapproximation using `np.argmax(np.sum(np.abs(mat[:, self.input_size:]), axis=0))` as refinement strategy; so far only epsilon equiv (`c5c8f02`)

In [1]:
import parse
import matplotlib.pyplot as plt
import numpy as np

In [2]:
benchmarks = [
    "../results/ACASXU_run2a_1_1_batch_2000-trunc-1010-0.05/",
    "../results/ACASXU_run2a_1_2_batch_2000-trunc-1010-0.05/",
    "../results/HAR-trunc-1000.1-0.25/",
    "../results/HAR-trunc-1000-0.25/",
    "../results/mnist_relu_3_100-trunc-400.1-0.25/",
    "../results/mnist_relu_3_100-trunc-400-0.25/",
    "../results/mnist8x8_student_12_12_12_10-9000.5-top/",
    "../results/mnist8x8_student_12_12_12_10-9100.5-top/",
    "../results/mnist8x8_student_24_12_10-9000.5-top/",
    "../results/mnist8x8_student_24_12_10-9100.5-top/",
    "../results/mnist8x8_student_36_10-9000.5-top/",
    "../results/mnist8x8_student_36_10-9100.5-top/"
]

In [3]:
pure_zono = {}
cegar_zono = {}
for b in benchmarks:
    try:
        pure_zono[b]=parse.BenchmarkRun(
            b+"1/nnequiv-091c775/stdout.log",
            b+"1/nnequiv-091c775/stderr.log",
            out_handlers=[parse.EquivLine],
            err_handlers=[parse.RunLim]
            )
    except FileNotFoundError:
        print(f"Could not find {b}")
        pure_zono[b]=None
    try:
        cegar_zono[b]=parse.BenchmarkRun(
            b+"1/nnequiv-c5c8f02/stdout.log",
            b+"1/nnequiv-c5c8f02/stderr.log",
            out_handlers=[parse.EquivLine],
            err_handlers=[parse.RunLim]
            )
    except FileNotFoundError:
        print(f"Could not find {b}")
        cegar_zono[b]=None

Could not find ../results/mnist_relu_3_100-trunc-400-0.25/
Could not find ../results/mnist8x8_student_12_12_12_10-9000.5-top/
Could not find ../results/mnist8x8_student_12_12_12_10-9100.5-top/
Could not find ../results/mnist8x8_student_24_12_10-9000.5-top/
Could not find ../results/mnist8x8_student_24_12_10-9100.5-top/
Could not find ../results/mnist8x8_student_36_10-9000.5-top/
Could not find ../results/mnist8x8_student_36_10-9100.5-top/


In [4]:
import analyse

In [5]:
comparison = analyse.RunlimComparator(pure_zono,cegar_zono,benchmarks)
    
comparison.render_table()

,Name,Status (1),Status (2),Real (1),Real (2),Time (1),Time (2),Mem (1),Mem (2)
0,../results/ACASXU_run2a_1_1_batch_2000-trunc-1010-0.05/,ok,ok,422.26,1185.97,421.64,1217.65,223.2,204.2
1,../results/ACASXU_run2a_1_2_batch_2000-trunc-1010-0.05/,ok,ok,661.97,1349.35,661.14,1391.36,292.8,222.7
2,../results/HAR-trunc-1000.1-0.25/,ok,ok,84.2,2.69,147.88,2.46,160.9,127.8
3,../results/HAR-trunc-1000-0.25/,out,out,10800.2,10800.1,17692,11797.4,416.5,386.1
4,../results/mnist_relu_3_100-trunc-400.1-0.25/,ok,ok,22.44,2.28,41.16,2.03,156.7,125.5
5,../results/mnist_relu_3_100-trunc-400-0.25/,None,out,nan,10800.1,nan,11507.6,nan,286.1
6,../results/mnist8x8_student_12_12_12_10-9000.5-top/,ok,None,75.52,nan,75.21,nan,116.7,nan
7,../results/mnist8x8_student_12_12_12_10-9100.5-top/,ok,None,242.38,nan,242.06,nan,119.9,nan
8,../results/mnist8x8_student_24_12_10-9000.5-top/,ok,None,41.22,nan,40.81,nan,115.5,nan
9,../results/mnist8x8_student_24_12_10-9100.5-top/,ok,None,119.91,nan,119.34,nan,117.1,nan
